# Helper functions

In [1]:
import csv
import re

path_to_shifts = "work_shifts.csv"
path_to_sales = "transactions.csv"

## Helper functions

# get access to the shifts hours
def loadShifts(path_to_csv):
    with open(path_to_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        count = 0
        for row in csv_reader:
            if count == 0:
                print("---")
            else:
                print(f"{row[0]}\t{row[1]}")
            count += 1

# convert break time from notes using regex
def getBreakTime(time):
    time = time.upper()
    time  = "".join(time.split())
    a,b  = time.split("-")
    # case 24 hours clock
    if (re.search("^(2[0-3]|[01]?[0-9]):([0-5]?[0-9])$", a) and re.search("^(2[0-3]|[01]?[0-9]):([0-5]?[0-9])$", b)):
        ha,ma = a.split(":")
        hb,mb = b.split(":")
        return (int(ha) * 60 + int(ma), int(hb) * 60 + int(mb))
    if (re.search("^(2[0-3]|[01]?[0-9])\.([0-5]?[0-9])$", a) and re.search("^(2[0-3]|[01]?[0-9])\.([0-5]?[0-9])$", b)):
        ha,ma = a.split(".")
        hb,mb = b.split(".")
        return (int(ha) * 60 + int(ma), int(hb) * 60 + int(mb))
    # case 12 hours clock
    # case AM or PM
    if (re.search("^(1[0-2]|0?[1-9])([AP][M])?$", a) and re.search("^(1[0-2]|0?[1-9])([AP][M])?$", b) ):
        if "AM" in a:
            a = int ( re.match("(1[0-2]|0?[1-9])", a).group(0) ) * 60
        else:
            a  = ( int ( re.match("(1[0-2]|0?[1-9])", a).group(0) ) + 12 ) * 60
        if "AM" in b:
            b = int ( re.match("(1[0-2]|0?[1-9])", b).group(0) ) * 60
        else:
            b = ( int ( re.match("(1[0-2]|0?[1-9])", b).group(0) ) + 12 ) * 60  
        return (a,b)
    if (re.search("^(2[0-3]|[01]?[0-9])$", a) and re.search("^(2[0-3]|[01]?[0-9])$", b)):
        return (int(a) * 60 , int(b) * 60)
    
    if (re.search("^(2[0-3]|[01]?[0-9])\.([0-5]?[0-9])$", a) or re.search("^(2[0-3]|[01]?[0-9])\.([0-5]?[0-9])$", b)):
        if "." in a:
            ha,ma = a.split(".")
        else:
            (ha,ma) = (a, 0)
        if "." in b:
            hb,mb = b.split(".")
        else:
            (hb,mb) = (b, 0)
        return ( (int(ha) + 12) * 60 + int(ma), (int(hb) + 12) * 60 + int(mb))
    
    if ( re.search("^(2[0-3]|[01]?[0-9])\.([0-5]?[0-9])([AP][M])?$", a) or re.search("^(2[0-3]|[01]?[0-9])\.([0-5]?[0-9])([AP][M])?$", b) ):
        a = re.sub("[AP][M]", "", a)
        b = re.sub("[AP][M]", "", b)
        if "." in a:
            ha,ma = a.split(".")
        else:
            (ha,ma) = (a, 0)
        if "." in b:
            hb,mb = b.split(".")
        else:
            (hb,mb) = (b, 0)
        return ( (int(ha) + 12) * 60 + int(ma), (int(hb) + 12) * 60 + int(mb))
    
    raise ValueError('Invalid range time')
    
# time format converters
def timeToMinutes(time):
    h,m = time.split(":")
    return int( int(h) * 60 + int(m) )

def timeToHours(time):
    h = time.split(":")[0]
    return int( int(h) * 60 )
    
# calculate work time for a range set
def calculateTime(range_time, shift_time, break_time):
    cursor  = range_time[0]
    total_time = 0
    while (cursor < range_time[1]):
        # case to take into account
        if (cursor >= range_time[0] and cursor <= range_time[1] 
           and cursor >= shift_time[0] and cursor < shift_time[1] 
            and (cursor < break_time[0] or cursor >= break_time[1])
           ):
            total_time += 1
        cursor += 1
    return total_time

# Solution

In [2]:
def process_shifts(path_to_csv):
    # create the base shifts vector
    result = dict( (f'{h}:00' ,  0.0) for (h) in range(9,23) )
    with open(path_to_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        count = 0
        for row in csv_reader:
            # 0 break_notes, 1 end_time, 2 pay_rate, 3 start_time
            if count == 0:
                count = 0
            else:
                shift_time = ( timeToMinutes(row[3]), timeToMinutes(row[1])) 
                break_time = getBreakTime(row[0])
                pay_rate_per_minutes  = float(row[2]) / 60.0 
                # iterate throw ranges and update result
                for pointer in range(shift_time[0], shift_time[1] , 60):
                    range_time = (pointer, pointer + 60)
                    result[
                        str(int( range_time[0] / 60 )) + ":00"
                    ] += pay_rate_per_minutes * calculateTime( range_time, shift_time, break_time ) 
            count += 1
    return result

def process_sales(path_to_csv):
    # create the base shifts vector
    result = dict( (f'{h}:00' ,  0.0) for (h) in range(9,23) )
    with open(path_to_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        count = 0
        for row in csv_reader:
            if count == 0:
                _
            else:
                # 0 amount,1 time
                # get  hour_time
                hour_time = row[1].split(":")[0] + ":00"
                ammount = float(row[0])
                # update the result
                result[ hour_time ] += ammount
            count += 1
    return result

def compute_percentage(shifts, sales):
    # create the base shifts vector
    result = dict( (f'{h}:00' ,  0.0) for (h) in range(9,23) )
    # iterate throw result
    for key in result:
        if (sales[key] == 0.0):
            result[key] = -shifts[key]
        else:
            result[key] = ( sales[key]/shifts[key] ) * 100 
    return result

def best_and_worst_hour(percentages):
    # find min and max thorw the percentages
    best = "9:00"
    worst = "9:00"
    for hour in percentages:
        if (percentages[best] < percentages[hour]):
            best = hour
        if (percentages[worst] > percentages[hour]):
            worst = hour
    return best, worst

# Debugging

In [3]:
shifts_processed = process_shifts(path_to_shifts)
sales_processed = process_sales(path_to_sales)
percentages = compute_percentage(shifts_processed, sales_processed)
best_hour, worst_hour = best_and_worst_hour(percentages)
(best_hour, worst_hour)

('18:00', '20:00')